In this notebook, you will learn to add AI options to a basic Teams-ai bot.

A Teams-ai application object can have an `ApplicationOptions.ai` property with `AIOption` type. A bot application with `ApplicationOptions.ai` property can work as a bot with AI abilities. For example, it can give a artificial response to your input according to prompts.

`AIOptions` consists of `planner`, `prompt` and `history`. The `prompt` determines what prompts to use. The `planner` will generate plan according to given prompts and chat history. We will show you how `prompt` and  `planner` work.

## Prompt
In this section, we will introduce how to generate a prompts folder as a prompts source base, which can be loaded by the `AIOption`.

Here is a sample code snippet about how to generate prompts source folder `src/prompts`, and a named prompt folder `src/prompts/chat` which consists of `config.json` and `skprompt.txt` as a `"chat"` prompt. 

In [ ]:
import os

# Define the file paths
prompts_folder_path = "src/prompts/"
prompt_name = "chat/"
config_file_path = f"{prompts_folder_path}{prompt_name}config.json"
skprompt_file_path = f"{prompts_folder_path}{prompt_name}skprompt.txt"
if not os.path.exists(prompts_folder_path):
    os.makedirs(prompts_folder_path)
if not os.path.exists(f"{prompts_folder_path}{prompt_name}"):
    os.makedirs(f"{prompts_folder_path}{prompt_name}")

# Define the config_json and skprompt_txt
config_json = """{
    "schema": 1,
    "description": "Chat with Teams Chef",
    "type": "completion",
    "completion": {
      "max_tokens": 150,
      "temperature": 0.9,
      "top_p": 0.0,
      "presence_penalty": 0.6,
      "frequency_penalty": 0.0,
      "stop_sequences": [
        "Human:",
        "AI:"
      ]
    }
}"""

skprompt_txt = """The following is a conversation with an AI assistant, its name is Teams Chef. 
Teams Chef is an expert in Microsoft Teams apps development and the Human is junior developer learning Microsoft Teams development for the first time. 
Teams Chef should always reply by explaining new concepts in simple terms using cooking as parallel concepts. 
Teams Chef should always greet the human, ask them their name, and then guide the junior developer in his journey to build new apps for Microsoft Teams.

{{$history}}
Human: {{$input}}
TeamsChef:"""

# Write the config_json to config.json file
with open(config_file_path, "w") as config_file:
    config_file.write(config_json)

# Write the skprompt_txt to skprompt.txt file
with open(skprompt_file_path, "w") as skprompt_file:
    skprompt_file.write(skprompt_txt)

# Print a success message
print(f"Files written successfully. Please check the files in the following paths:\n{config_file_path}\n{skprompt_file_path}")

## Planner
In this section, we will introduce how does a `planner` generates a plan.

1. First, we import needed packages

In [ ]:
# import modules from src
import src.config as config
from src.bot import *
from teams import AIHistoryOptions, AzureOpenAIPlanner, AzureOpenAIPlannerOptions, ConversationHistory, ConversationState, TempState, UserState
from botbuilder.schema import ChannelAccount, ConversationAccount
from unittest.mock import MagicMock 

2. Second, we initiate a `planner` object with Azure OpenAI keys and values, determining prompt folder.

In [ ]:
# register a planner
config.AZURE_OPENAI_KEY = 'b9e28116eeff4b129db5155fd261a541' # Azure OpenAI API key
config.AZURE_OPENAI_MODEL_DEPLOYMENT_NAME = 'gpt-35-turbo-zihch' # Azure OpenAI model deployment name
config.AZURE_OPENAI_ENDPOINT = 'https://azure-openai-zihch.openai.azure.com/' # Azure OpenAI endpoint
planner = AzureOpenAIPlanner(
    AzureOpenAIPlannerOptions(
        config.AZURE_OPENAI_KEY,
        config.AZURE_OPENAI_MODEL_DEPLOYMENT_NAME,
        config.AZURE_OPENAI_ENDPOINT,
        prompt_folder="src/prompts",
    )
)

3. Then, we mock a `TurnContext` object and a `TurnState` object. They will be used when generating a plan.

    > In this example, we set `TempState.input='hi'` to emulate that user inputs 'hi' as the input text.

In [ ]:

# mock the TurnContext object
context=MagicMock(spec=TurnContext)

# mock the TurnState object
temp=TempState(input='hi', history='', output='')
user=UserState(__key__='mock_key', channel=ChannelAccount())
conversation=ConversationState(
    __key__='mock_key',
    account=ConversationAccount(),
    channel=ChannelAccount(),
    history=ConversationHistory()
)
state=TurnState(
    temp=temp,
    user=user,
    conversation=conversation
)

print(f"context={context}")
print(f"state={state}")

4. Last, we call `planner._prompt_manager.render_prompt()` and `planner.generate_plan()` to see what prompt content passed to planner and what plan is generated by planner. 

    > In this example, we choose `"chat"` as the prompt template name to generate a prompt.

In [ ]:

# render prompt
prompt="chat"
prompt_templates = await planner._prompt_manager.render_prompt(context, state, prompt)
print(f"prompt_templates:\n{prompt_templates}")

# generate plan
plan = await planner.generate_plan(
    context, state, prompt, history_options=AIHistoryOptions(assistant_history_type="text")
)
print(f"plan:\n{plan}")

## Add AIOptions to a bot
Now we know how to manage the prompts folder and how to set up a planner. We can initiate a Teams-ai bot application with `AIOptions` with `prompt` and `planner` we need.

In [ ]:
from botbuilder.core import BotFrameworkAdapterSettings, MemoryStorage
from teams import AIOptions, Application, ApplicationOptions, TurnState
storage = MemoryStorage()
app = Application[TurnState](
    ApplicationOptions(
        auth=BotFrameworkAdapterSettings(
            app_id=config.app_id,
            app_password=config.app_password,
        ),
        ai=AIOptions(
            planner=planner,
            prompt="chat",
            history=AIHistoryOptions(assistant_history_type="text"),
        ),
        storage=storage,
    )
)